In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# !rm -rf sample_data
# !cp /content/drive/My\ Drive/imdb-dataset-of-50k-movie-reviews.zip /content
# !unzip imdb-dataset-of-50k-movie-reviews.zip
# !rm -rf imdb-dataset-of-50k-movie-reviews.zip

In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords

from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
import matplotlib.pyplot as plt
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
# from keras import initializers, regulariz

Using TensorFlow backend.


In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
data_df = pd.read_csv('IMDB Dataset.csv')
data_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
data_df.shape

(50000, 2)

In [7]:
data_df['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [0]:
# def clean_data(data_df):

#     # To lowercase
#     data_df['review'] = data_df['review'].apply(lambda x: ' '.join(word.lower() for word in str(x).split()))

#     # Remove HTML tags
#     data_df['review'] = data_df['review'].apply(lambda x: re.sub(r'<[^>]*>', ' ', str(x)))

#     # Remove stop words
#     data_df['review'] = data_df['review'].apply(lambda x: ' '.join(word for word in str(x).split() if word not in stop_words))

#     # Retain only alphabets
#     data_df['review'] = data_df['review'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', str(x)))

#     # Remov 1 and 2 length characters
#     # data_df['review'] = data_df['review'].apply(lambda x: ' '.join(word for word in str(x).split() if len(word) > 2))


#     return data_df

import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

In [9]:
data_df['review'] = data_df['review'].apply(lambda x: clean_text(x))
data_df.head()

,review,sentiment
0,one reviewer ha mention watch 1 oz episode you...,positive
1,wonderful little production br br film techniq...,positive
2,think wa wonderful way spend time hot summer w...,positive
3,basically family little boy jake think zombie ...,negative
4,petter matteis love time money visually stun f...,positive


In [10]:
data_df.loc[data_df['sentiment'] == 'positive', 'sentiment'] = int(1)
data_df.loc[data_df['sentiment'] == 'negative', 'sentiment'] = int(0)
data_df.head()

,review,sentiment
0,one reviewer ha mention watch 1 oz episode you...,1
1,wonderful little production br br film techniq...,1
2,think wa wonderful way spend time hot summer w...,1
3,basically family little boy jake think zombie ...,0
4,petter matteis love time money visually stun f...,1


In [0]:
X, Y = data_df['review'], data_df['sentiment']

In [0]:
Y = pd.get_dummies(Y)

In [0]:
# Spit data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.10, random_state = 42)

In [14]:
print (X_train.shape)
print (Y_train.shape)
print (X_test.shape)
print (Y_test.shape)

(45000,)
(45000, 2)
(5000,)
(5000, 2)


In [15]:
# integer encode the documents
vocab_size = 6000
encoded_docs = [one_hot(d, vocab_size) for d in X_train]

# pad documents to a max length of 300 words
max_length = 256
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[2169 5302 4155 ...    0    0    0]
 [1133 1005 5864 ...    0    0    0]
 [3155  351 1985 ...    0    0    0]
 ...
 [2697 4476 5486 ...    0    0    0]
 [1708  530 5219 ...    0    0    0]
 [1286 1985 4217 ...    0    0    0]]


In [20]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_length))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.02))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.02))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.02))
# model.add(Dense(512, activation='relu'))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(2, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
# print(model.summary())
# fit the model
model.fit(padded_docs, Y_train, epochs = 6, batch_size=100)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, Y_train)
print(f'Accuracy: {accuracy*100}')

Epoch 1/6
45000/45000 [==============================] - 524s 12ms/step - loss: 0.4187 - acc: 0.8076
Epoch 2/6
45000/45000 [==============================] - 521s 12ms/step - loss: 0.2744 - acc: 0.8898
Epoch 3/6
45000/45000 [==============================] - 510s 11ms/step - loss: 0.2261 - acc: 0.9129
Epoch 4/6
45000/45000 [==============================] - 501s 11ms/step - loss: 0.1826 - acc: 0.9336
Epoch 5/6
45000/45000 [==============================] - 499s 11ms/step - loss: 0.1384 - acc: 0.9530
Epoch 6/6
45000/45000 [==============================] - 569s 13ms/step
Accuracy: 98.49


In [0]:
encoded_docs_test = [one_hot(d, vocab_size) for d in X_test]
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')

In [25]:
print (f'Metrics: {model.metrics_names}')

Metrics: ['loss', 'acc']


In [26]:
model.evaluate(padded_docs_test, Y_test)

5000/5000 [==============================] - 66s 13ms/step


[0.4102548369154334, 0.8598]